# Regression Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from linearReg import LinearReg
from gdOptimizers import GDOptimizers

### Setup of data

In [ ]:
def franke(x, y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [ ]:
def data(n = 100):
    x = np.random.rand(n) #an array of n unordered uniform random numbers from 0 to 1
    y = np.random.rand(n)
    xy = np.column_stack((x, y))
    
    noise = np.random.randn(n) / 10
    z = franke(x, y) + noise
    return xy, z

In [ ]:
xy, z = data()
xy_train, xy_test, z_train, z_test = train_test_split(xy, z, test_size = 0.2)

### FFNN

In [ ]:
from FFNN import FFNN

nn = FFNN(inputs = 2)
nn.addLayer(neurons = 5, activation = "sigmoid")
nn.addLayer(neurons = 5, activation = "sigmoid")
nn.addLayer(neurons = 1, activation = "sigmoid")

nn.feedForward(xy_train)

In [ ]:
th = np.array([[[1,2], [3,4]]])
print(th)
th = th.tolist()
th.append([[5,6], [7,8]])
print(th)
print(th[0])

### Ridge regression with SGD

In [ ]:
model = LinearReg(order = 3, lmda = 1E-5)
sgd = GDOptimizers(learning_rate = 0.01, epochs = 1000, batch_size = 10, optimizer = "SGD")
z_train_mdl = model.fit(xy_train, z_train, sgd)

z_mdl = model.predict(xy_test)
print(f"Mean squared error on training data = {mean_squared_error(z_train_mdl, z_train):.4f}")
print(f"Mean squared error = {mean_squared_error(z_mdl, z_test):.4f}")

In [ ]:
model = LinearReg(order = 3, lmda = 1E-5)
momentum = GDOptimizers(learning_rate = 0.01, epochs = 1000, batch_size = 10, optimizer = "momentum", alpha=0.5)
z_train_mdl = model.fit(xy_train, z_train, momentum)

z_mdl = model.predict(xy_test)
print(f"Mean squared error on training data = {mean_squared_error(z_train_mdl, z_train):.4f}")
print(f"Mean squared error = {mean_squared_error(z_mdl, z_test):.4f}")

In [ ]:
# visual representation of grid search
# uses seaborn heatmap, you can also do this with matplotlib imshow
import seaborn as sns

sns.set()

eta_vals = np.logspace(-4, -2, 6)
lmbd_vals = np.logspace(-5, 1, 7)
train_mse = np.zeros((len(eta_vals), len(lmbd_vals)))
test_mse = np.zeros((len(eta_vals), len(lmbd_vals)))

for i, eta in enumerate(eta_vals):
    for j, lmda in enumerate(lmbd_vals):
        model = LinearReg(order = 3, lmda = lmda)
        sgd = GDOptimizers(learning_rate = eta, epochs = 100, batch_size = 10, optimizer = "SGD")
        
        z_train_mdl = model.fit(xy_train, z_train, sgd)
        z_mdl = model.predict(xy_test)

        train_mse[i][j] = mean_squared_error(z_train_mdl, z_train)
        test_mse[i][j] = mean_squared_error(z_mdl, z_test)

        
fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(train_mse, annot=True, ax=ax, cmap="viridis_r", xticklabels=np.log10(lmbd_vals), yticklabels=np.log10(eta_vals))
ax.set_title("Training MSE")
ax.set_ylabel("$\eta$")
ax.set_xlabel("$\lambda$")
plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(test_mse, annot=True, ax=ax, cmap="viridis_r", xticklabels=np.log10(lmbd_vals), yticklabels=np.log10(eta_vals))
ax.set_title("Test MSE")
ax.set_ylabel("$\eta$")
ax.set_xlabel("$\lambda$")
plt.show()